# Anime_Score_Sim
Purpose, to generate anime that are similar to other anime by cosine similarity. Compare the  results of cosine similarity via score, via user scaled score, and via anime scaled score. 
This outputs .csv files. 



In [1]:
import pandas as pd

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from scipy.sparse import csr_matrix

In [3]:
csvtoload = 'mal_scores_train_nonzero_v2.csv'# this is a subset of scores used for EDA and to design the system. Replace with a full csv, or to an sql database for the full dataset. 

In [4]:
ratings = pd.read_csv(csvtoload) # This is a subset of reviews with all 0's dropped. 
ratings['userid']=ratings['index']
ratings['score']=ratings['animescore']

In [5]:
ratings1 = ratings[ratings['score']!=0] # this is a sanity check to ensure no 0's got into the system. 

In [6]:
ratings1.head()

,Unnamed: 0,Unnamed: 0.1,animeid,animescore,index,name,status,unkey,userid,score,useravg,animeavg,score_anime_scaled,score_usr_scaled
0,0,13202,2001,10,Lancerevomr,Lancerevomr,COMPLETED,2001Lancerevomr,Lancerevomr,10,10.0,8.681996,1.318004,0.0
1,1,21990,10719,10,kuelpanda,kuelpanda,COMPLETED,10719kuelpanda,kuelpanda,10,9.5,7.535472,2.464528,0.5
2,2,21991,10155,9,kuelpanda,kuelpanda,COMPLETED,10155kuelpanda,kuelpanda,9,9.5,7.105895,1.894105,-0.5
3,3,22409,15051,10,vxxyman,vxxyman,COMPLETED,15051vxxyman,vxxyman,10,10.0,7.654129,2.345871,0.0
4,4,23792,1,10,pinheiro,pinheiro,COMPLETED,1pinheiro,pinheiro,10,10.0,8.804023,1.195977,0.0


In [7]:
ratings1.head()# checking our data

,Unnamed: 0,Unnamed: 0.1,animeid,animescore,index,name,status,unkey,userid,score,useravg,animeavg,score_anime_scaled,score_usr_scaled
0,0,13202,2001,10,Lancerevomr,Lancerevomr,COMPLETED,2001Lancerevomr,Lancerevomr,10,10.0,8.681996,1.318004,0.0
1,1,21990,10719,10,kuelpanda,kuelpanda,COMPLETED,10719kuelpanda,kuelpanda,10,9.5,7.535472,2.464528,0.5
2,2,21991,10155,9,kuelpanda,kuelpanda,COMPLETED,10155kuelpanda,kuelpanda,9,9.5,7.105895,1.894105,-0.5
3,3,22409,15051,10,vxxyman,vxxyman,COMPLETED,15051vxxyman,vxxyman,10,10.0,7.654129,2.345871,0.0
4,4,23792,1,10,pinheiro,pinheiro,COMPLETED,1pinheiro,pinheiro,10,10.0,8.804023,1.195977,0.0


In [8]:
#anime_pivot_table = ratings1.pivot_table(index='userid', columns = 'animeid',values='score_scaled')
# a pivot table  pandas function would be easier than what I do here but will run most systems out of memory. 

In [9]:
users = ratings.userid.unique()
animes = ratings.animeid.unique()

In [10]:
userid2idx = {o:i for i,o in enumerate(users)} # turn the user strings into a numerical value. 
animeid2idx = {o:i for i,o in enumerate(animes)} # remove missing anime numbers and re-order 

In [11]:
ratings1['anime_id'] = ratings1.animeid.apply(lambda x: animeid2idx[x]) #the above code set up mapping, this applies it. 
ratings1.userid = ratings1.userid.apply(lambda x: userid2idx[x])# the above code set up mapping, this applies it. 


We are using these mapping to generate which row and column each username and anime_id will go into. 

In [12]:


user_u = list(sorted(ratings1.animeid.unique()))
anime_id_u = list(sorted(ratings1.userid.unique()))

data = ratings1['score'].tolist() # Changing this line allows us to test score vs scaled score vs user scaled score
row = ratings1.userid.astype('category')
col = ratings1.anime_id.astype('category')
sparse_matrix = csr_matrix((data, (row, col)), shape=(len(users), len(animes)))

In [13]:
reviewmatrix =sparse_matrix.T 

In [14]:
#col

In [15]:
#row

In [16]:
score_cos =cosine_similarity(reviewmatrix) # now we generatie our cosine of similarity

In [17]:
#animeinfo= pd.read_csv('animeinfo3.csv')
#animenames=pd.merge(ratings1,animeinfo,left_on='anime_id',right_on='animeid') # 

In [18]:
#ratings1.sort_values('anime_id')

In [19]:
inv_map = {v: k for k, v in  animeid2idx.items()} # we have to make an inverse map of the which row/column number is what anime number.


In [20]:
anime_names = pd.read_csv('Data/animeinfo3.csv').set_index('animeid')['name'].to_dict() # Then we map out anime_id to anime name

In [21]:
anime_names[32878] # confirming that our mapping is correct by going to https://myanimelist.net/anime/32878

'ēlDLIVE'

In [22]:
anime_in_order = []

In [23]:
for  anime in list(inv_map.values()): # we are going through the inverse map, and making a list of the anime names in the row/column order. 
    try:
        anime_in_order.append(anime)
    except:
        anime_in_order.append('unknownanime'+str(anime)) # just in case an anime that I haven't gotten info on gets a review. 

In [24]:
anime_in_order

[2001,
 10719,
 10155,
 15051,
 1,
 8074,
 11757,
 14741,
 5525,
 16011,
 10379,
 966,
 29786,
 2167,
 7193,
 226,
 11061,
 24833,
 263,
 33,
 12079,
 22535,
 28977,
 5114,
 12291,
 19815,
 23289,
 21,
 11321,
 30276,
 479,
 1275,
 440,
 30346,
 22693,
 31240,
 16498,
 15097,
 31173,
 4181,
 9253,
 4483,
 9513,
 28623,
 1735,
 15315,
 30307,
 12967,
 223,
 7592,
 22199,
 20,
 1535,
 18679,
 32686,
 1604,
 813,
 3470,
 32182,
 28755,
 6702,
 23283,
 3841,
 31680,
 32281,
 269,
 32962,
 20507,
 5391,
 777,
 20785,
 31338,
 14813,
 210,
 31245,
 412,
 6033,
 30240,
 1530,
 5262,
 3269,
 32995,
 2923,
 889,
 227,
 33089,
 27631,
 10620,
 8841,
 29803,
 47,
 33836,
 32587,
 2604,
 15809,
 28497,
 1827,
 1571,
 16067,
 31964,
 1639,
 3603,
 22877,
 384,
 28223,
 22043,
 22319,
 8516,
 1520,
 10079,
 10397,
 13759,
 270,
 21639,
 14753,
 2896,
 11001,
 33926,
 20583,
 15335,
 355,
 1210,
 6547,
 4898,
 32801,
 22101,
 28171,
 7054,
 19,
 16524,
 32935,
 1657,
 5530,
 24277,
 13601,
 12445,
 2

In [25]:
rev1_sim_cos_df = pd.DataFrame(score_cos, index=anime_in_order , columns=anime_in_order)

In [56]:
rev1_sim_cos_df.head()

,2001,10719,10155,15051,1,8074,11757,14741,5525,16011,...,29283,17825,25623,34852,25597,29361,25871,30468,22493,14207
2001,1.000000,0.262517,0.149805,0.153384,0.406437,0.339543,0.361834,0.292844,0.117722,0.179530,...,0.002520,0.0,0.0,0.0,0.0,0.005670,0.0,0.0,0.0,0.0
10719,0.262517,1.000000,0.261582,0.221942,0.167863,0.378653,0.367601,0.450414,0.111146,0.308265,...,0.008814,0.0,0.0,0.0,0.0,0.005876,0.0,0.0,0.0,0.0
10155,0.149805,0.261582,1.000000,0.138804,0.086168,0.201043,0.185011,0.215420,0.133285,0.227624,...,0.019353,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
15051,0.153384,0.221942,0.138804,1.000000,0.099055,0.162902,0.212074,0.277650,0.056914,0.137558,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.406437,0.167863,0.086168,0.099055,1.000000,0.246013,0.259522,0.196765,0.093725,0.101847,...,0.006247,0.0,0.0,0.0,0.0,0.006247,0.0,0.0,0.0,0.0


In [26]:
rev1_sim_cos_df.to_csv('reviewsim_prod.csv')

In [27]:
rev1_sim_cos_df.loc[:,'Girls und Panzer'].sort_values()[-20:]

KeyError: 'the label [Girls und Panzer] is not in the [columns]'

In [ ]:
rev1_sim_cos_df.loc[:,'Death Note'].sort_values()[-20:]

In [ ]:
rev1_sim_cos_df.loc[:,'Bakemonogatari'].sort_values()[-20:]

In [ ]:
rev1_sim_cos_df.loc[:,'Sword Art Online'].sort_values()[-20:]

In [ ]:
rev1_sim_cos_df.loc[:,'Sword Art Online'].sort_values()[-20:]

In [28]:


user_u = list(sorted(ratings1.animeid.unique()))
anime_id_u = list(sorted(ratings1.userid.unique()))

data = ratings1['score_anime_scaled'].tolist() # Changing this line allows us to test score vs scaled score vs user scaled score
row = ratings1.userid.astype('category')
col = ratings1.anime_id.astype('category')
sparse_matrix = csr_matrix((data, (row, col)), shape=(len(users), len(animes)))

In [29]:
reviewmatrix =sparse_matrix.T 

In [30]:
#col

In [31]:
#row

In [32]:
score_cos =cosine_similarity(reviewmatrix) # now we generatie our cosine of similarity

In [33]:
#animeinfo= pd.read_csv('animeinfo3.csv')
#animenames=pd.merge(ratings1,animeinfo,left_on='anime_id',right_on='animeid') # 

In [34]:
#ratings1.sort_values('anime_id')

In [35]:
inv_map = {v: k for k, v in  animeid2idx.items()} # we have to make an inverse map of the which row/column number is what anime number.


In [36]:
anime_names = pd.read_csv('Data/animeinfo3.csv').set_index('animeid')['name'].to_dict() # Then we map out anime_id to anime name

In [37]:
anime_names[32878] # confirming that our mapping is correct by going to https://myanimelist.net/anime/32878

'ēlDLIVE'

In [38]:
anime_in_order = []

In [39]:
for  anime in list(inv_map.values()): # we are going through the inverse map, and making a list of the anime names in the row/column order. 
    try:
        anime_in_order.append(anime)
    except:
        anime_in_order.append('unknownanime'+str(anime)) 

In [40]:
anime_in_order

[2001,
 10719,
 10155,
 15051,
 1,
 8074,
 11757,
 14741,
 5525,
 16011,
 10379,
 966,
 29786,
 2167,
 7193,
 226,
 11061,
 24833,
 263,
 33,
 12079,
 22535,
 28977,
 5114,
 12291,
 19815,
 23289,
 21,
 11321,
 30276,
 479,
 1275,
 440,
 30346,
 22693,
 31240,
 16498,
 15097,
 31173,
 4181,
 9253,
 4483,
 9513,
 28623,
 1735,
 15315,
 30307,
 12967,
 223,
 7592,
 22199,
 20,
 1535,
 18679,
 32686,
 1604,
 813,
 3470,
 32182,
 28755,
 6702,
 23283,
 3841,
 31680,
 32281,
 269,
 32962,
 20507,
 5391,
 777,
 20785,
 31338,
 14813,
 210,
 31245,
 412,
 6033,
 30240,
 1530,
 5262,
 3269,
 32995,
 2923,
 889,
 227,
 33089,
 27631,
 10620,
 8841,
 29803,
 47,
 33836,
 32587,
 2604,
 15809,
 28497,
 1827,
 1571,
 16067,
 31964,
 1639,
 3603,
 22877,
 384,
 28223,
 22043,
 22319,
 8516,
 1520,
 10079,
 10397,
 13759,
 270,
 21639,
 14753,
 2896,
 11001,
 33926,
 20583,
 15335,
 355,
 1210,
 6547,
 4898,
 32801,
 22101,
 28171,
 7054,
 19,
 16524,
 32935,
 1657,
 5530,
 24277,
 13601,
 12445,
 2

In [41]:
anime_rev_sim_cos_df = pd.DataFrame(score_cos, index=anime_in_order , columns=anime_in_order)
anime_rev_sim_cos_df.to_csv('anime_review_scaled_prod.csv')

In [ ]:
anime_rev_sim_cos_df.loc[:,'Girls und Panzer'].sort_values()[-20:]

In [ ]:
rev1_sim_cos_df.loc[:,'Death Note'].sort_values()[-20:]

In [ ]:
anime_rev_sim_cos_df.loc[:,'Bakemonogatari'].sort_values()[-20:]

In [ ]:
rev1_sim_cos_df.loc[:,'Sword Art Online'].sort_values()[-20:]

In [ ]:
rev1_sim_cos_df.loc[:,'Sword Art Online'].sort_values()[-20:]

In [42]:


user_u = list(sorted(ratings1.animeid.unique()))
anime_id_u = list(sorted(ratings1.userid.unique()))

data = ratings1['score_usr_scaled'].tolist() # Changing this line allows us to test score vs scaled score vs user scaled score
row = ratings1.userid.astype('category')
col = ratings1.anime_id.astype('category')
sparse_matrix = csr_matrix((data, (row, col)), shape=(len(users), len(animes)))

In [43]:
reviewmatrix =sparse_matrix.T 

In [44]:
#col

In [45]:
#row

In [46]:
score_cos =cosine_similarity(reviewmatrix) # now we generatie our cosine of similarity

In [47]:
#animeinfo= pd.read_csv('animeinfo3.csv')
#animenames=pd.merge(ratings1,animeinfo,left_on='anime_id',right_on='animeid') # 

In [48]:
#ratings1.sort_values('anime_id')

In [49]:
inv_map = {v: k for k, v in  animeid2idx.items()} # we have to make an inverse map of the which row/column number is what anime number.


In [50]:
anime_names = pd.read_csv('Data/animeinfo3.csv').set_index('animeid')['name'].to_dict() # Then we map out anime_id to anime name

In [51]:
anime_names[32878] # confirming that our mapping is correct by going to https://myanimelist.net/anime/32878

'ēlDLIVE'

In [52]:
anime_in_order = []

In [53]:
for  anime in list(inv_map.values()): # we are going through the inverse map, and making a list of the anime names in the row/column order. 
    try:
        anime_in_order.append(anime)
    except:
        anime_in_order.append('unknownanime'+str(anime)) 

In [54]:
anime_in_order

[2001,
 10719,
 10155,
 15051,
 1,
 8074,
 11757,
 14741,
 5525,
 16011,
 10379,
 966,
 29786,
 2167,
 7193,
 226,
 11061,
 24833,
 263,
 33,
 12079,
 22535,
 28977,
 5114,
 12291,
 19815,
 23289,
 21,
 11321,
 30276,
 479,
 1275,
 440,
 30346,
 22693,
 31240,
 16498,
 15097,
 31173,
 4181,
 9253,
 4483,
 9513,
 28623,
 1735,
 15315,
 30307,
 12967,
 223,
 7592,
 22199,
 20,
 1535,
 18679,
 32686,
 1604,
 813,
 3470,
 32182,
 28755,
 6702,
 23283,
 3841,
 31680,
 32281,
 269,
 32962,
 20507,
 5391,
 777,
 20785,
 31338,
 14813,
 210,
 31245,
 412,
 6033,
 30240,
 1530,
 5262,
 3269,
 32995,
 2923,
 889,
 227,
 33089,
 27631,
 10620,
 8841,
 29803,
 47,
 33836,
 32587,
 2604,
 15809,
 28497,
 1827,
 1571,
 16067,
 31964,
 1639,
 3603,
 22877,
 384,
 28223,
 22043,
 22319,
 8516,
 1520,
 10079,
 10397,
 13759,
 270,
 21639,
 14753,
 2896,
 11001,
 33926,
 20583,
 15335,
 355,
 1210,
 6547,
 4898,
 32801,
 22101,
 28171,
 7054,
 19,
 16524,
 32935,
 1657,
 5530,
 24277,
 13601,
 12445,
 2

In [55]:

user_rev_sim_cos_df = pd.DataFrame(score_cos, index=anime_in_order , columns=anime_in_order)
user_rev_sim_cos_df.to_csv('user_review_scaled_prod.csv')

In [ ]:
user_rev_sim_cos_df.loc[:,'Girls und Panzer'].sort_values()[-20:]

In [ ]:
rev1_sim_cos_df.loc[:,'Death Note'].sort_values()[-20:]

In [ ]:
user_rev_sim_cos_df.loc[:,'Bakemonogatari'].sort_values()[-20:]

In [ ]:
rev1_sim_cos_df.loc[:,'Sword Art Online'].sort_values()[-20:]

In [ ]:
rev1_sim_cos_df.loc[:,'Sword Art Online'].sort_values()[-20:]